<a href="https://colab.research.google.com/github/PierreLeveau/automl/blob/main/notebooks/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification Using AutoML

In this notebook, we will see how we can simply create an image classification model with AutoML to pre-annotate our dataset on the [Kili Platform](https://cloud.kili-technology.com/label/).

## Install

We first follow the install procedure explained in the [README.md](https://github.com/kili-technology/automl/blob/main/README.md). 

In [ ]:
!git clone https://github.com/kili-technology/automl.git

In [ ]:
%cd automl


Install the packages. This should take less than a minute. 

In [ ]:
%%capture
!git submodule update --init
!pip install -r requirements.txt -r kiliautoml/utils/ultralytics/yolov5/requirements.txt
!pip install -e .
!pip install kili

## Imports

In [ ]:
import os
from getpass import getpass
from tqdm.autonotebook import tqdm

from kili.client import Kili

Setup the python PATH to use kiliautoml.

In [ ]:
KILI_URL="https://cloud.kili-technology.com/"
os.environ["PYTHONPATH"] += ":/content/automl/"

After getting your API key from the Kili platform, you can setup your environment variables.

In [ ]:
api_key = getpass("Add your API Key here: ")
api_endpoint = f"{KILI_URL}api/label/v2/graphql" # If you are not using Kili SaaS, change the endpoint to your configuration

## Setup a mock Kili project

Setup the kili connection.

In [ ]:
kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

### Create the project

In [ ]:
json_interface = {
    "jobRendererWidth": 0.2,
    "jobs": {
        "CLASSIFICATION_JOB": {
            "mlTask": "CLASSIFICATION",
            "content": {
                "categories": {
                    "VEHICLE": {
                        "name": "vehicle"
                    },
                    "NON_VEHICLE": {
                        "name": "non vehicle"
                    }
                },
                "input": "radio"
            },
            "required": 0,
            "isChild": False,
            "instruction": "Class of vehicle presence"
        }
    }
}

In [ ]:
project_id = kili.create_project(
        title="Vehicle Classification",
        description="Classify vehicle presence",
        input_type="IMAGE",
        json_interface=json_interface
)["id"]

### Add assets

In [ ]:
vehicle_assets = [
    {
        "externalId": f"{i}",
        "content": f"https://storage.googleapis.com/kili-machine-learning-automl/notebooks/vehicle_classification/vehicles/image_{i}.png",
        "metadata": {}
    }
    for i in range(500)
]
non_vehicle_assets = [
    {
        "externalId": f"{len(vehicle_assets) + i}",
        "content": f"https://storage.googleapis.com/kili-machine-learning-automl/notebooks/vehicle_classification/non-vehicles/image_{i}.png",
        "metadata": {}
    }
    for i in range(500)
]
assets_to_import = vehicle_assets + non_vehicle_assets

Now we send the data to our Kili project.

In [ ]:
external_id_array = [a.get("externalId") for a in assets_to_import]
content_array = [a.get("content") for a in assets_to_import]
json_metadata_array = [a.get("metadata") for a in assets_to_import]
kili.append_many_to_dataset(project_id=project_id, 
                            content_array=content_array,
                            external_id_array=external_id_array, 
                            json_metadata_array=json_metadata_array)

### Add labels to assets

We add labels to half of the data to simulate a project where we haven't labeled much data and we want to predict the labels of the unlabeled data. 

In [ ]:
asset_ids = kili.assets(project_id=project_id, fields=["id", "externalId"], first=1000)

for asset_id in tqdm(asset_ids):
    external_id = int(asset_id["externalId"])
    if external_id < 300:
        kili.append_to_labels(label_asset_id=asset_id["id"],
                              json_response={
                                  "CLASSIFICATION_JOB": {
                                      "categories": [{"name": "VEHICLE"}]
                                  }
                              })

    elif 499 < external_id < 800:
        kili.append_to_labels(label_asset_id=asset_id["id"],
                              json_response={
                                  "CLASSIFICATION_JOB": {
                                      "categories": [{"name": "NON_VEHICLE"}]
                                  }
                              })
    else:
        pass

You can now click on the following link to see the assets in your project:

In [ ]:
print(f"{KILI_URL}label/projects/{project_id}/menu/queue?currentPage=1&pageSize=20")

## Training a image classifier with Kiliautoml

The following command will automatically download the labeled data in your Kili project. Then, it will choose the right model for your task, train it with this data and save it locally.

In [ ]:
!kiliautoml train \
    --api-key {api_key} \
    --project-id {project_id} \
    --epochs 30

### Send predictions

Now we can use our local trained model to predict the classes of our image assets and send the prediction scores to the project on Kili. These preannotations can then be validated or corrected by annotators.

In [ ]:
!kiliautoml predict \
    --api-key {api_key} \
    --project-id {project_id}

Now you can ckeck that your assets have predictions on [Kili](https://cloud.kili-technology.com/)!

In [ ]:
print(f"{KILI_URL}label/projects/{project_id}/menu/queue?currentPage=1&pageSize=20")

### Label Errors

You can add wrong labels to a small portion of the data to simulate a project where some data is incorrectly labeled. 


In [ ]:
asset_ids = kili.assets(project_id=project_id, fields=["id", "externalId"])

for asset_id in tqdm(asset_ids):
    external_id = int(asset_id["externalId"])
    if 299 < external_id < 310:
        kili.append_to_labels(label_asset_id=asset_id["id"],
                              json_response={
                                  "CLASSIFICATION_JOB": {
                                      "categories": [{"name": "NON_VEHICLE"}]
                                  }
                              })

    elif 799 < external_id < 810:
        kili.append_to_labels(label_asset_id=asset_id["id"],
                              json_response={
                                  "CLASSIFICATION_JOB": {
                                      "categories": [{"name": "VEHICLE"}]
                                  }
                              })
    else:
        pass

This command analyses the labeled data to detect potential labeling errors and sends warnings to the concerned assets. The user can then use the `potential_label_error` filter on the project's asset exploration to find the potentially problematic assets.

In [ ]:
!kiliautoml label_errors \
    --api-key {api_key} \
    --project-id {project_id}

You can see in the project that several uncorrectly labeled images have been spotted.

In [ ]:
print(f"{KILI_URL}label/projects/{project_id}/menu/queue?currentPage=1&metadata%5Blabeling_error%5D=true&pageSize=20")